In [ ]:
%pip install torchgeometry

In [ ]:
from torchgeometry.losses import one_hot
import os
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import time
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import Resize, PILToTensor, ToPILImage, Compose, InterpolationMode
import segmentation_models_pytorch as smp
import wandb

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
%pip install segmentation-models-pytorch

In [ ]:


model = smp.UnetPlusPlus(
    encoder_name="resnet34",        
    encoder_weights="imagenet",     
    in_channels=3,                  
    classes=3     
)

# Parameters

In [ ]:
num_classes = 3
epochs = 50
learning_rate = 1e-03
batch_size = 4
display_step = 25

#checkpoint_path = '/kaggle/working/unet_model.pth'
checkpoint_path = '/model/unet.pth'

loss_epoch_array = []
train_accuracy = []
test_accuracy = []
valid_accuracy = []

# Dataloader

In [7]:
transform = Compose([Resize((256, 256), interpolation=InterpolationMode.BILINEAR),
                     PILToTensor()])

In [8]:
class UNetDataClass(Dataset):
    def __init__(self, images_path, masks_path, transform):
        super(UNetDataClass, self).__init__()
        
        images_list = os.listdir(images_path)
        masks_list = os.listdir(masks_path)
        
        images_list = [images_path + image_name for image_name in images_list]
        masks_list = [masks_path + mask_name for mask_name in masks_list]
        
        self.images_list = images_list
        self.masks_list = masks_list
        self.transform = transform
        
    def __getitem__(self, index):
        img_path = self.images_list[index]
        mask_path = self.masks_list[index]
        
        # Open image and mask
        data = Image.open(img_path)
        label = Image.open(mask_path)
        
        # Normalize
        data = self.transform(data) / 255
        label = self.transform(label) / 255
        
        label = torch.where(label>0.65, 1.0, 0.0)
        
        label[2, :, :] = 0.0001
        label = torch.argmax(label, 0).type(torch.int64)
        
        return data, label
    
    def __len__(self):
        return len(self.images_list)

In [ ]:
#images_path = "/kaggle/input/bkai-igh-neopolyp/train/train/"
#masks_path =  "/kaggle/input/bkai-igh-neopolyp/train_gt/train_gt/"
images_path = "/train/train/"
masks_path =  "/train_gt/train_gt/"

In [10]:
unet_dataset = UNetDataClass(images_path, masks_path, transform)

In [11]:
train_size = 0.9
valid_size = 0.1

In [12]:
train_set, valid_set = random_split(unet_dataset, 
                                    [int(train_size * len(unet_dataset)) , 
                                     int(valid_size * len(unet_dataset))])

In [13]:
train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_set, batch_size=batch_size, shuffle=True)

# Data Augmentation

In [ ]:
from albumentations import (
    Compose,
    HorizontalFlip,
    VerticalFlip,
    RandomGamma,
    RGBShift,
)

augmentation = Compose([
    HorizontalFlip(p=0.5),
    VerticalFlip(p=0.5),
    RandomGamma (gamma_limit=(70, 130), eps=None, always_apply=False, p=0.2),
    RGBShift(p=0.3, r_shift_limit=10, g_shift_limit=10, b_shift_limit=10),
])

In [ ]:
class SegDataClass(Dataset):
    def __init__(self, images_path, masks_path, transform=None, augmentation=None):
        super(SegDataClass, self).__init__()
        
        images_list = os.listdir(images_path)
        masks_list = os.listdir(masks_path)
        
        images_list = [os.path.join(images_path, image_name) for image_name in images_list]
        masks_list = [os.path.join(masks_path, mask_name) for mask_name in masks_list]
        
        self.images_list = images_list
        self.masks_list = masks_list
        self.transform = transform
        self.augmentation = augmentation
        
    def __getitem__(self, index):
        img_path = self.images_list[index]
        mask_path = self.masks_list[index]
        
        data = Image.open(img_path)
        label = Image.open(mask_path)
        
        if self.augmentation:
            augmented = self.augmentation(image=np.array(data), mask=np.array(label))
            data = Image.fromarray(augmented['image'])
            label = Image.fromarray(augmented['mask'])
        
        data = self.transform(data) / 255
        label = self.transform(label) / 255
        
        label = torch.where(label > 0.65, 1.0, 0.0)
        label[2, :, :] = 0.0001
        label = torch.argmax(label, 0).type(torch.int64)
        
        return data, label
    
    def __len__(self):
        return len(self.images_list)

aug_dataset = SegDataClass(images_path, masks_path, transform=transform, augmentation=augmentation)

In [17]:
train_aug_set, valid_aug_set = random_split(aug_dataset, 
                                    [int(train_size * len(aug_dataset)) , 
                                     int(valid_size * len(aug_dataset))])

In [ ]:
train_dataloader = DataLoader(train_aug_set, batch_size=batch_size, shuffle=True)

# Loss function

In [ ]:
class CEDiceLoss(nn.Module):
    def __init__(self, weights) -> None:
        super(CEDiceLoss, self).__init__()
        self.eps: float = 1e-6
        self.weights: torch.Tensor = weights

    def forward(
            self,
            input: torch.Tensor,
            target: torch.Tensor) -> torch.Tensor:
        if not torch.is_tensor(input):
            raise TypeError("Input type is not a torch.Tensor. Got {}"
                            .format(type(input)))
        if not len(input.shape) == 4:
            raise ValueError("Invalid input shape, we expect BxNxHxW. Got: {}"
                             .format(input.shape))
        if not input.shape[-2:] == target.shape[-2:]:
            raise ValueError("input and target shapes must be the same. Got: {}"
                             .format(input.shape, input.shape))
        if not input.device == target.device:
            raise ValueError(
                "input and target must be in the same device. Got: {}" .format(
                    input.device, target.device))
        if not self.weights.shape[1] == input.shape[1]:
            raise ValueError("The number of weights must equal the number of classes")
        if not torch.sum(self.weights).item() == 1:
            raise ValueError("The sum of all weights must equal 1")
            
        # cross entropy loss
        celoss = nn.CrossEntropyLoss(self.weights)(input, target)
        
        # compute softmax over the classes axis
        input_soft = F.softmax(input, dim=1)

        # create the labels one hot tensor
        target_one_hot = one_hot(target, num_classes=input.shape[1],
                                 device=input.device, dtype=input.dtype)

        # compute the actual dice score
        dims = (2, 3)
        intersection = torch.sum(input_soft * target_one_hot, dims)
        cardinality = torch.sum(input_soft + target_one_hot, dims)

        dice_score = 2. * intersection / (cardinality + self.eps)
        
        dice_score = torch.sum(dice_score * self.weights, dim=1)
        
        return torch.mean(1. - dice_score) + celoss

# Training

**Initialize weights**

In [20]:
def weights_init(model):
    if isinstance(model, nn.Linear):
        # Xavier Distribution
        torch.nn.init.xavier_uniform_(model.weight)

**Train model**

In [ ]:
weights = torch.Tensor([[0.4, 0.55, 0.05]]).cuda()
loss_function = CEDiceLoss(weights)

optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)

# Learning rate scheduler
#learing_rate_scheduler = lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.6)
learing_rate_scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
#learing_rate_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.2, patience=5)

In [21]:
# Train function for each epoch
def train(train_dataloader, valid_dataloader,learing_rate_scheduler, epoch, display_step):
    lr = optimizer.param_groups[0]["lr"]
    print(f"Start epoch #{epoch+1}, learning rate for this epoch: {lr}")
    start_time = time.time()
    train_loss_epoch = 0
    test_loss_epoch = 0
    last_loss = 999999999
    model.train()
    for i, (data,targets) in enumerate(train_dataloader):
        
        # Load data into GPU
        data, targets = data.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(data)

        # Backpropagation, compute gradients
        loss = loss_function(outputs, targets.long())
        loss.backward()

        # Apply gradients
        optimizer.step()
        
        # Save loss
        train_loss_epoch += loss.item()
        if (i+1) % display_step == 0:
#             accuracy = float(test(test_loader))
            print('Train Epoch: {} [{}/{} ({}%)]\tLoss: {:.4f}'.format(
                epoch + 1, (i+1) * len(data), len(train_dataloader.dataset), 100 * (i+1) * len(data) / len(train_dataloader.dataset), 
                loss.item()))
                  
    print(f"Done epoch #{epoch+1}, time for this epoch: {time.time()-start_time}s")
    train_loss_epoch/= (i + 1)
    
    # Evaluate the validation set
    model.eval()
    with torch.no_grad():
        for data, target in valid_dataloader:
            data, target = data.to(device), target.to(device)
            test_output = model(data)
            test_loss = loss_function(test_output, target)
            test_loss_epoch += test_loss.item()
            
    test_loss_epoch/= (i+1)
    
    return train_loss_epoch , test_loss_epoch

In [ ]:
def test(dataloader):
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for i, (data, targets) in enumerate(dataloader):
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            _, pred = torch.max(outputs, 1)
            test_loss += targets.size(0)
            correct += torch.sum(pred == targets).item()
    return 100.0 * correct / test_loss

In [ ]:
model.apply(weights_init)
model.to(device)

In [ ]:
wandb.login(
    key = "key_here",
)
wandb.init(
    project="bkai-polyp",
    name="exponential_lr2",
)

In [26]:
# Training loop
train_loss_array = []
test_loss_array = []
last_loss = 9999999999999
for epoch in range(epochs):
    train_loss_epoch = 0
    test_loss_epoch = 0
    (train_loss_epoch, test_loss_epoch) = train(train_dataloader, 
                                              valid_dataloader, 
                                              learing_rate_scheduler, epoch, display_step)
    
    if test_loss_epoch < last_loss:
        torch.save(model.state_dict(), checkpoint_path)
        last_loss = test_loss_epoch

    wandb.log({"Train loss": train_loss_epoch, "Valid loss": test_loss_epoch, "Learning rate": optimizer.param_groups[0]["lr"]})
    learing_rate_scheduler.step()
    train_loss_array.append(train_loss_epoch)
    test_loss_array.append(test_loss_epoch)

Start epoch #1, learning rate for this epoch: 0.001
Train Epoch: 1 [100/900 (11.11111111111111%)]	Loss: 1.2312
Train Epoch: 1 [200/900 (22.22222222222222%)]	Loss: 1.4080
Train Epoch: 1 [300/900 (33.333333333333336%)]	Loss: 1.1388
Train Epoch: 1 [400/900 (44.44444444444444%)]	Loss: 1.0587
Train Epoch: 1 [500/900 (55.55555555555556%)]	Loss: 1.1722
Train Epoch: 1 [600/900 (66.66666666666667%)]	Loss: 1.2157
Train Epoch: 1 [700/900 (77.77777777777777%)]	Loss: 1.1907
Train Epoch: 1 [800/900 (88.88888888888889%)]	Loss: 1.0988
Train Epoch: 1 [900/900 (100.0%)]	Loss: 1.5023
Done epoch #1, time for this epoch: 65.09183645248413s
Start epoch #2, learning rate for this epoch: 0.00095
Train Epoch: 2 [100/900 (11.11111111111111%)]	Loss: 1.1014
Train Epoch: 2 [200/900 (22.22222222222222%)]	Loss: 0.9473
Train Epoch: 2 [300/900 (33.333333333333336%)]	Loss: 1.1420
Train Epoch: 2 [400/900 (44.44444444444444%)]	Loss: 1.2612
Train Epoch: 2 [500/900 (55.55555555555556%)]	Loss: 1.2555
Train Epoch: 2 [600/900

In [28]:
import gc

# Collect garbage and clear cache
gc.collect()
torch.cuda.empty_cache()

In [ ]:
#model.load_state_dict(torch.load(checkpoint_path, weights_only=True))
model.to("cpu")

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


<All keys matched successfully>

**Create submission**

In [63]:
class UNetTestDataClass(Dataset):
    def __init__(self, images_path, transform):
        super(UNetTestDataClass, self).__init__()
        
        images_list = os.listdir(images_path)
        images_list = [images_path+i for i in images_list]
        
        self.images_list = images_list
        self.transform = transform
        
    def __getitem__(self, index):
        img_path = self.images_list[index]
        data = Image.open(img_path)
        h = data.size[1]
        w = data.size[0]
        data = self.transform(data) / 255        
        return data, img_path, h, w
    
    def __len__(self):
        return len(self.images_list)

In [ ]:
#path = '/kaggle/input/bkai-igh-neopolyp/test/test/'
path = '/test/test/'
unet_test_dataset = UNetTestDataClass(path, transform)
test_dataloader = DataLoader(unet_test_dataset, batch_size=8, shuffle=True)

In [ ]:
model.eval()
#if not os.path.isdir("/kaggle/working/predicted_masks"):
#    os.mkdir("/kaggle/working/predicted_masks")
for _, (img, path, h, w) in enumerate(test_dataloader):
    with torch.no_grad():
        predicted_mask = model(img)
    for i in range(len(path)):
        image_id = path[i].split('/')[-1].split('.')[0]
        filename = image_id + ".png"
        mask2img = Resize((h[i].item(), w[i].item()), interpolation=InterpolationMode.NEAREST)(ToPILImage()(F.one_hot(torch.argmax(predicted_mask[i], 0)).permute(2, 0, 1).float()))
        mask2img.save(os.path.join("/test_gt/test_gt", filename))

In [ ]:
def rle_to_string(runs):
    return ' '.join(str(x) for x in runs)

def rle_encode_one_mask(mask):
    pixels = mask.flatten()
    pixels[pixels > 0] = 255
    use_padding = False
    if pixels[0] or pixels[-1]:
        use_padding = True
        pixel_padded = np.zeros([len(pixels) + 2], dtype=pixels.dtype)
        pixel_padded[1:-1] = pixels
        pixels = pixel_padded
    
    rle = np.where(pixels[1:] != pixels[:-1])[0] + 2
    if use_padding:
        rle = rle - 1
    rle[1::2] = rle[1::2] - rle[:-1:2]
    return rle_to_string(rle)

def mask2string(dir):
    ## mask --> string
    strings = []
    ids = []
    ws, hs = [[] for i in range(2)]
    for image_id in os.listdir(dir):
        id = image_id.split('.')[0]
        path = os.path.join(dir, image_id)
        print(path)
        img = cv2.imread(path)[:,:,::-1]
        h, w = img.shape[0], img.shape[1]
        for channel in range(2):
            ws.append(w)
            hs.append(h)
            ids.append(f'{id}_{channel}')
            string = rle_encode_one_mask(img[:,:,channel])
            strings.append(string)
    r = {
        'ids': ids,
        'strings': strings,
    }
    return r


MASK_DIR_PATH = '/kaggle/working/predicted_masks'
MASK_DIR_PATH = '/test_gt/test_gt/'
res = mask2string(MASK_DIR_PATH)
df = pd.DataFrame(columns=['Id', 'Expected'])
df['Id'] = res['ids']
df['Expected'] = res['strings']
df.to_csv(r'output.csv', index=False)

/kaggle/working/predicted_masks/998906d3694abb47953b0e4909384b57.png
/kaggle/working/predicted_masks/b70dd094a7f32574d6c748c41743c6c0.png
/kaggle/working/predicted_masks/97e1c0e9082ea2c193ac8d551c149b60.png
/kaggle/working/predicted_masks/d6bf62f215f0da4ad3a7ab8df9da7386.png
/kaggle/working/predicted_masks/3c3ca4d5060a633a8d5b2b2b55157b77.png
/kaggle/working/predicted_masks/fe1f119f21b248d152b672ab3492fc62.png
/kaggle/working/predicted_masks/0626ab4ec3d46e602b296cc5cfd263f1.png
/kaggle/working/predicted_masks/e2cd066b9fdbc3bbc04a3afe1f119f21.png
/kaggle/working/predicted_masks/8fa8625605da2023387fd56c04414eaa.png
/kaggle/working/predicted_masks/c656702fa602bb3c7abacdbd7e6afd56.png
/kaggle/working/predicted_masks/2d9e593b6be1ac29adbe86f03d900fd1.png
/kaggle/working/predicted_masks/eb1ef57af2ed9fbb63b28163a745959c.png
/kaggle/working/predicted_masks/5e8f14e1e0ae936de314f2d95e6c487f.png
/kaggle/working/predicted_masks/4e8bfb905b78a91391adc0bb223c4eaf.png
/kaggle/working/predicted_masks/e5